# Behavior cloning flow

Create the full flow for training a model for behavior cloning. This notebook programmatically generates a new set of exp/runs that cover all the necessary components for a behavior cloning system (sensor processing, behavior cloning training and verification notebooks)

It writes the exp/runs into an external directory fully separated from the github source, and creates an automation script that runs them. A separate directory for results is also created. 

Finally, it runs the necessary notebooks to execute the whole flow using papermill.

The results directory contain the output of this flow, both in terms of trained models, as well as results (in the verification exp/run).

In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import socket
import pathlib
import yaml
import tqdm
import papermill
# from automate import automate_exprun
import bc_factory 
from demonstration.demonstration import list_demos

# Setting up the separate directory
Setting up a separate directory for generated exp/run config files and the results. This cell will create a new directory. 

__Note__
Currently, as of 2025-09-03, we don't have a way to put the training data into this directory. Which means that once the code below creates the new directories, we need to manually copy the demonstrations as follows:
* the demonstration descriptor (eg. {demonstration_run}.yaml) into {directory_name}/exprun/demonstration
* the demonstration runs into {directory_name}/result/demonstration/{demonstration_run}/

It would be useful to set up a special import mechanism for this.

In [ ]:
# Setting up a separate directory for generated exp/run config files and the results

# the most likely changing names.
directory_name = "BerryPicker-BC"
demonstration_cam = "dev0"
demonstration_run = "touch-apple"

hostname = socket.gethostname()
print(f"***Hostname is {hostname}")
if hostname == "raven":
    rootdir = pathlib.Path(f"~/WORK/_DataExternal/").expanduser()
elif hostname == "szenes.local":
    rootdir = pathlib.Path(f"~/Documents/Develop/Data/").expanduser()
elif hostname == "glassy":
    rootdir = pathlib.Path(f"~/Work/_DataExternal/").expanduser()
else:
    rootdir = pathlib.Path(Config()["experiment_external"])

exprun_path, result_path = bc_factory.external_setup(directory_name, rootdir)

In [ ]:
def generate_sensorprocessing_conv_vae(exprun_path, result_path, params):
    """Generate the experiment for the conv-vae sensorprocessing with the right training data and parameters. Returns a dictionary with the experiment, runname as well as an entry that will be used for the automation. """
    val = {}
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]
    val["save_period"] = 5
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]
    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), params["exp_sp"], params["run_sp"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "TrainVAE"
    v["notebook"] = "sensorprocessing/Train-Conv-VAE.ipynb"
    vparams = {}
    vparams["experiment"] = params["exp_sp"]
    vparams["run"] = params["run_sp"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    return {"experiment": params["exp_sp"], "runname": params["run_sp"], "automation_entry": v}

In [ ]:
def generate_bc_train(exprun_path, result_path, params):
    """Generate the experiment for the training of behavior cloning with the right training data and parameters. At the same time, it also generates the corresponding bcrun experiment, which will be used to drive the robot.  
    """    
    val = {}
    val["name"] = params["name"]
    val["exp_sp"] = params["exp_sp"]
    val["run_sp"] = params["run_sp"]
    # these had not yet been studied for alternatives
    val["sequence_length"] = params["sequence_length"]
    val["loss"] = params["loss"]
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]

    if params["algorithm"] == "lstm-mdn": 
        val["exp_mdn"] = "behavior_cloning"
        val["run_mdn"] = "mdn_for_bc_00"
        val["controller"] = "bc_LSTM_MDN"
        val["controller_file"] = "controller.pth"
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "lstm":
        val["controller"] = "bc_LSTM"
        val["controller_file"] = "controller.pth"
        val["num_layers"] = params["num_layers"]
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "lstm-residual":
        val["controller"] = "bc_LSTM_Residual"
        val["controller_file"] = "controller.pth"
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "mlp":
        val["controller"] = "bc_MLP"
        val["controller_file"] = "controller.pth"
        val["hidden_layers"] = params["hidden_layers"]
        val["hidden_layer_1"] = params["hidden_layer_1"]
        val["hidden_layer_2"] = params["hidden_layer_2"]
        
    else:
        raise Exception(f"*** generate_behaviorcloning: algorithm {params['algorithm']} not yet supported")
    
    val["control_size"] = 6 
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]

    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), params["exp_bc"], params["run_bc"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "BehaviorCloning"
    v["notebook"] = "behavior_cloning/Train_BehaviorCloning.ipynb"
    vparams = {}
    vparams["run"] = params["run_bc"]
    vparams["experiment"] = params["exp_bc"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    # now, generate the bcrun experiment file, which is the same with the "runbc_" prefix
    # FIXME: these need to be configurable
    valrun = {}
    valrun["name"] = f"generated_run_{params['run_bc']}"
    valrun["exp_robot_controller"] = "robot_al5d"
    valrun["run_robot_controller"] = "position_controller_00"
    valrun["exp_camera_controller"] = "controllers"
    valrun["run_camera_controller"] = "camera_cam0_controller"
    valrun["control_camera"] = "dev0"
    valrun["exp_bc"] = "behavior_cloning"
    valrun["run_bc"] = "bc_lstm_0001"

    path = pathlib.Path(Config().get_experiment_path(), params["exp_bc"], "runbc_"+ params["run_bc"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(valrun, f)

    return {"experiment": params["exp_bc"], "runname": params["run_bc"], "automation_entry": v}

In [ ]:
def generate_bc_verify(exprun_path, result_path, params):
    """Generate the experiment for the behavior cloning with the right training data and parameters. At the same time, it also generates the corresponding bcrun experiment, which will be used to drive the robot.  
    """    
    val = {}
    val["exp_sp"] = params["exp_sp"]
    val["run_sp"] = params["run_sp"]
    val["exp_bc"] = params["exp_bc"]
    val["run_bc"] = params["run_bc"]
    val["exp_bc_verify"] = params["exp_bc_verify"]
    val["run_bc_verify"] = params["run_bc_verify"]
    val["verification_data"] = params["verification_data"]

    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), params["exp_bc"], params["run_bc"] + "_verify.yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "BehaviorCloning"
    v["notebook"] = "behavior_cloning/Verify_BehaviorCloning.ipynb"
    vparams = {}
    vparams["run"] = params["run_bc_verify"]
    vparams["experiment"] = params["exp_bc_verify"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    return {"experiment": params["exp_bc_verify"], "runname": params["run_bc_verify"], "automation_entry": v}

### Generate a range of exp/runs to be run

define which demonstrations will be used for:
* training the sensor processing (sp_training_data)
* training the behavior cloning (bc_training_data)
* verifying the behavior cloning (bc_verification_data) 
   etc.

In [ ]:
expruns = []

# overall values
latent_size = 128

# FIXME: set the training data through some data structure passed as parameter....
# setting the training data
cam = "dev0"
#run_demo = "touch-apple"
experiment = "demonstration"
# run = "freeform"
# run = "random-both-cameras"
# run = "random-both-cameras-video"
exp = Config().get_experiment(experiment, demonstration_run)
demos = list_demos(exp)
print("***The demos considered")
pprint.pprint(demos)

data = []
for demo in demos:
    data.append([demonstration_run, demo, demonstration_cam])

training_data = data[0:-2]
sp_training_data = training_data
bc_training_data = training_data

# setting the validation data
validation_data = data[-2:]
sp_validation_data = validation_data
bc_validation_data = validation_data
# experiment, let's run the verification
# also on the data which was present in the training
bc_verification_data = data

# determine these values based on experience
epochs_sp = 300
epochs_bc = 500

# generate sensor processing expruns
exp_sp = "sensorprocessing_conv_vae"
run_sp = "sp_conv_vae_0001"
params = {}
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["epochs"] = epochs_sp
params["training_data"] = sp_training_data
params["validation_data"] = sp_validation_data
exprun = generate_sensorprocessing_conv_vae(
    exprun_path = exprun_path, result_path = result_path, params = params)
expruns.append(exprun)

# generate the behavior cloning expruns to match with the sensor processing components from above

exp_bc = "behavior_cloning"

# an MLP model
params = {}
params["name"] = "MLP 2 hidden" # should be descriptive
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_mlp_0001"
params["algorithm"] = "mlp"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_layers"] = 2
params["hidden_layer_1"] = 50
params["hidden_layer_2"] = 20
params["epochs"] = epochs_bc
params["sequence_length"] = 1
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the MLP model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "bc_mlp_0001_verify"
paramsv["verification_data"] = bc_verification_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)


# a pure LSTM model
params = {}
params["name"] = "LSTM 2 layers"
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_lstm_0001"
params["algorithm"] = "lstm"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["num_layers"] = 2
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_verification_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the LSTM model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "bc_lstm_0001_verify"
paramsv["verification_data"] = bc_verification_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)

# a residual LSTM model
params = {}
params["name"] = "LSTM residual"
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_lstm_residual_0001"
params["algorithm"] = "lstm-residual"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the LSTM model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "bc_lstm_residual_0001_verify"
paramsv["verification_data"] = bc_verification_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)

# an LSTM-MDN based model
params = {}
params["name"] = "LSTM MDN"
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_lstm_mdn_0001"
params["algorithm"] = "lstm-mdn"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the LSTM-MDN model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "bc_lstm_mdn_0001_verify"
paramsv["verification_data"] = bc_verification_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)


### Generate an automation

Generate an exp/run of the type "automate" based on the list expruns. The individual automation entries should be already in the "automation_entry" field. The only thing that is really decided here is the "creation_style", which can be exist-ok or discard-old, if we want to start this from scratch. 

In [ ]:
value = {}
val = []
value["exps_to_run"] = val

# Use exist_ok not to-re-run previously successfully run models
creation_style = "exist-ok"
# creation_style = "discard-old"

for exprun in expruns:
    v = exprun["automation_entry"]
    if "verify" in v["params"]["run"]:
        v["params"]["creation_style"] = "discard-old"
    else:
        v["params"]["creation_style"] = creation_style
    val.append(v)

path = pathlib.Path(Config().get_experiment_path(), "automate", "flow_bc.yaml")
with open(path, "w") as f:
    yaml.dump(value, f)

### Run the automation script

Run the automation script, as a series of notebooks. In order to follow the execution inside these notebooks, one needs to open the output notebook, which is in the output_filename. 

* FIXME: this should be done such that I can also run it from command line

In [ ]:
experiment = "automate"
run = "flow_bc"
exp = Config().get_experiment(experiment, run)

print(f"***Starting automated running of the flow.\n The path for the output notebooks is\n{result_path}")

for item in tqdm.tqdm(exp["exps_to_run"]):
    print(f"***Automating {item['notebook']} :\n {item['params']['experiment']}/{item['params']['run']}")
    notebook_path = pathlib.Path("..", item["notebook"])
    output_filename = f"{notebook_path.stem}_{item['params']['experiment']}_{item['params']['run']}_output{notebook_path.suffix}"
    print(f"--> {output_filename}")
    # pprint.pprint(item["params"])
    output_path = pathlib.Path(result_path, output_filename)
    try:
        papermill.execute_notebook(
            notebook_path,
            output_path.absolute(),
            cwd=notebook_path.parent,
            parameters=item["params"]
        )
    except Exception as e:
        print(f"There was an exception {e}")    